In [1]:
# Reload the notebook if an external file is updated
%load_ext autoreload
%autoreload 2

import sys

from pathlib import Path

path = str(Path.cwd().parent)
sys.path.append(path)

In [2]:
import numpy as np

from constant import SETTINGS
from copy import deepcopy
from datatype.dataset import Dataset
from datatype.settings import Settings
from datatype.spectrogram import Linear, Mel, Spectrogram
from io import BytesIO
from itertools import permutations
from math import sqrt
from PIL import Image, ImageDraw, ImageEnhance, ImageFilter, ImageFont, ImageOps

In [3]:
def create_grid(collection, method):
    length = len(collection)
    root = sqrt(length)
    root = int(root)
    
    column = row = root
    
    width, height = np.vectorize(lambda x: x.size)(collection)
    width = np.amax(width)
    height = np.amax(height)

    border = 50
    offset = 10
    
    grid = Image.new(
        'L', 
        color='white',
        size=(
            (column * width) + offset, 
            (row * height) + (offset * 2)
        )
    )
    
    for index, image in enumerate(collection, 0):
        grid.paste(
            image, 
            box=(
                (index % column * width) + offset, 
                (index // column * height) + (offset * 2)
            )
        )
        
    grid = ImageOps.expand(
        grid, 
        border=border, 
        fill=(255)
    )
    
    draw = ImageDraw.Draw(grid)
    font = ImageFont.truetype("FONTS/arial.ttf", 24)
    
    w, h = grid.size

    draw.text(
        (w // 2, 10),
        method,
        (0),
        font=font
    )
    
    return grid

In [4]:
def create_image(spectrogram):
    # spectrogram[
    #     np.where(spectrogram > [100])
    # ] = [255]
        
    buffer = BytesIO()
    image = Image.fromarray(~spectrogram)
    
    # median = ImageFilter.MaxFilter(size=3)
    # image = image.filter(median)

    return image

In [5]:
def create_spectrogram(signal, settings):
    spectrogram = Spectrogram()
    strategy = Mel(signal, settings)
    spectrogram.strategy = strategy

    return spectrogram.generate()

In [6]:
dataset = Dataset('segment')
dataframe = dataset.load()

In [7]:
path = SETTINGS.joinpath('dereverberate.json')
dereverberate = Settings.from_file(path)

In [8]:
path = SETTINGS.joinpath('spectrogram.json')
settings = Settings.from_file(path)

In [9]:
# minimum = 10
# maximum = minimum + 10

# subset = dataframe.iloc[minimum:maximum]
subset = dataframe.sample(200)

In [10]:
callback = {
    'dereverberate': np.frompyfunc(
        lambda x: x.dereverberate(dereverberate),
        1,
        0
    ),
    
    'normalize': np.frompyfunc(
        lambda x: x.normalize(),
        1,
        0
    ),
    
    'reduce': np.frompyfunc(
        lambda x: x.reduce(),
        1,
        0
    )
}

In [13]:
keys = [*callback]

methods = []

for i in range(0, len(keys) + 1):
    for permutation in permutations(keys, i):
        if permutation:
            methods.append(permutation)
            
len(list(methods))

15

In [ ]:
for index, method in enumerate(methods, 0):
    index = str(index).zfill(2)
    base = '_'.join(method)
    filename = f"{index}_{base}.png"
    
    segment = deepcopy(
        subset.segment.to_numpy()
    )
    
    for function in method:        
        callback[function](segment)

    spectrogram = np.frompyfunc(
        lambda x, y: create_spectrogram(x, y),
        2,
        1
    )(segment, settings)

    image = np.frompyfunc(
        lambda x: create_image(x),
        1,
        1
    )(spectrogram)

    grid = create_grid(image, base)
    grid.save(filename, format='png')